# Computer Languages (R, Python, Julia, C)

## Types of computer languages

* **Compiled languages** (low-level languages): C/C++, Fortran, ... 
  - Directly compiled to machine code that is executed by CPU 
  - Pros: fast, memory efficient
  - Cons: longer development time, hard to debug

* **Interpreted language** (high-level languages): R, Matlab, Python, SAS IML, JavaScript, ... 
  - Interpreted by interpreter
  - Pros: fast prototyping
  - Cons: excruciatingly slow for loops

* Mixed (dynamic) languages: Matlab (JIT), R (`compiler` package), Julia, Cython, JAVA, ...
  - Pros and cons: between the compiled and interpreted languages

* Script languages: Linux shell scripts, Perl, ...
  - Extremely useful for some data preprocessing and manipulation

* Database languages: SQL, Hive (Hadoop).  
  - Data analysis *never* happens if we do not know how to retrieve data from databases  
  
  [203B (Introduction to Data Science)](https://ucla-biostat203b-2020winter.github.io/schedule.html) covers some basics on scripting and database.

## How high-level languages work?

- Typical execution of a high-level language such as R, Python, and Matlab.

<img src="./r-bytecode.png" align="center" width="400"/>

- To improve efficiency of interpreted languages such as R or Matlab, conventional wisdom is to avoid loops as much as possible. Aka, **vectorize code**
> The only loop you are allowed to have is that for an iterative algorithm.

- When looping is unavoidable, need to code in C, C++, or Fortran. This creates the notorious **two language problem**  
Success stories: the popular `glmnet` package in R is coded in Fortran; `tidyverse` and `data.table` packages use a lot RCpp/C++.

<img src="./two_language_problem.jpg" align="center" width="600"/>

- High-level languages have made many efforts to bring themselves closer to the performance of low-level languages such as C, C++, or Fortran, with a variety levels of success.   
    - Matlab has employed JIT (just-in-time compilation) technology since 2002.  
    - Since R 3.4.0 (Apr 2017), the JIT bytecode compiler is enabled by default at its level 3.   
    - Cython is a compiler system based on Python.  

- Modern languages such as Julia tries to solve the two language problem. That is to achieve efficiency without vectorizing code.

- Julia execution.

<img src="./julia_compile.png" align="center" width="400"/>

<img src="./julia_introspect.png" align="center" width="400"/>

## Gibbs sampler example by Doug Bates

- Doug Bates (member of R-Core, author of popular R packages `Matrix` and `lme4`)
    
    > As some of you may know, I have had a (rather late) mid-life crisis and run off with another language called Julia.   
    >
    > -- <cite>Doug Bates (on the `knitr` Google Group)</cite>

- An example from Dr. Doug Bates's slides [Julia for R Programmers](http://www.stat.wisc.edu/~bates/JuliaForRProgrammers.pdf).

- The task is to create a Gibbs sampler for the density  
$$
f(x, y) = k x^2 exp(- x y^2 - y^2 + 2y - 4x), x > 0
$$
using the conditional distributions
$$
\begin{eqnarray*}
  X | Y &\sim& \Gamma \left( 3, \frac{1}{y^2 + 4} \right) \\
  Y | X &\sim& N \left(\frac{1}{1+x}, \frac{1}{2(1+x)} \right).
\end{eqnarray*}
$$

* R solution. The `RCall.jl` package allows us to execute R code without leaving the `Julia` environment. We first define an R function `Rgibbs()`.

In [1]:
using RCall

R"""
library(Matrix)

Rgibbs <- function(N, thin) {
  mat <- matrix(0, nrow=N, ncol=2)
  x <- y <- 0
  for (i in 1:N) {
    for (j in 1:thin) {
      x <- rgamma(1, 3, y * y + 4) # 3rd arg is rate
      y <- rnorm(1, 1 / (x + 1), 1 / sqrt(2 * (x + 1)))
    }
    mat[i,] <- c(x, y)
  }
  mat
}
"""

RObject{ClosSxp}
function (N, thin) 
{
    mat <- matrix(0, nrow = N, ncol = 2)
    x <- y <- 0
    for (i in 1:N) {
        for (j in 1:thin) {
            x <- rgamma(1, 3, y * y + 4)
            y <- rnorm(1, 1/(x + 1), 1/sqrt(2 * (x + 1)))
        }
        mat[i, ] <- c(x, y)
    }
    mat
}


To generate a sample of size 10,000 with a thinning of 500. How long does it take?

In [2]:
R"""
system.time(Rgibbs(10000, 500))
"""

RObject{RealSxp}
   user  system elapsed 
 25.976   1.782  28.269 


* This is a Julia function for the simple Gibbs sampler:

In [3]:
using Distributions

function jgibbs(N, thin)
    mat = zeros(N, 2)
    x = y = 0.0
    for i in 1:N
        for j in 1:thin
            x = rand(Gamma(3, 1 / (y * y + 4)))
            y = rand(Normal(1 / (x + 1), 1 / sqrt(2(x + 1))))
        end
        mat[i, 1] = x
        mat[i, 2] = y
    end
    mat
end

jgibbs (generic function with 1 method)

Generate the same number of samples. How long does it take?

In [4]:
jgibbs(100, 5); # warm-up
@elapsed jgibbs(10000, 500)

0.411166215

We see 40-80 fold speed up of `Julia` over `R` on this example, **with similar coding effort**!

## Comparing Julia, R, Python, and C

To better understand how these languages work, we use a simple example: summing a vector.

### Julia

`@time`, `@elapsed`, `@allocated` macros:

In [5]:
using Random # standard library
Random.seed!(123) # seed
x = rand(1_000_000) # 1 million random numbers in [0, 1)

@time sum(x) # first run includes compilation time

  0.035964 seconds (76.55 k allocations: 3.836 MiB)


500060.34072352527

In [6]:
@time sum(x) # no compilation time after first run

  0.000565 seconds (1 allocation: 16 bytes)


500060.34072352527

In [7]:
# just the runtime
@elapsed sum(x)

0.00073795

In [8]:
# just the allocation
@allocated sum(x)

16

Use package `BenchmarkTools.jl` for more robust benchmarking. Analog of `microbenchmark` or `bench` package in R.

In [9]:
using BenchmarkTools

bm = @benchmark sum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     274.836 μs (0.00% GC)
  median time:      444.936 μs (0.00% GC)
  mean time:        496.362 μs (0.00% GC)
  maximum time:     3.650 ms (0.00% GC)
  --------------
  samples:          9915
  evals/sample:     1

In [10]:
using Statistics # standard library
benchmark_result = Dict() # a dictionary to store median runtime (in milliseconds)
benchmark_result["Julia builtin"] = median(bm.times) / 1e6

0.444936

### C

We would use the low-level C code as the baseline for copmarison. In Julia, we can easily run compiled C code using the `ccall` function.

In [11]:
using Libdl

C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -std=c99 -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [12]:
# make sure it gives same answer
c_sum(x)

500060.340723512

In [13]:
bm = @benchmark c_sum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.221 ms (0.00% GC)
  median time:      1.500 ms (0.00% GC)
  mean time:        1.551 ms (0.00% GC)
  maximum time:     7.370 ms (0.00% GC)
  --------------
  samples:          3199
  evals/sample:     1

In [14]:
# store median runtime (in milliseconds)
benchmark_result["C"] = median(bm.times) / 1e6

1.500243

### R, buildin sum

Next we compare to the build in `sum` function in R, which is implemented using C.

In [15]:
using RCall

R"""
library(microbenchmark)
y <- $x
rbm <- microbenchmark(sum(y))
"""

RObject{VecSxp}
Unit: milliseconds
   expr      min      lq   mean   median       uq      max neval
 sum(y) 1.238856 1.29021 1.4202 1.341087 1.485662 2.365598   100


In [16]:
# store median runtime (in milliseconds)
@rget rbm # dataframe
benchmark_result["R builtin"] = median(rbm[!, :time]) / 1e6

1.341087

### R, handwritten loop

Handwritten loop in R is much slower.

In [17]:
using RCall

R"""
sum_r <- function(x) {
  s <- 0
  for (xi in x) {
    s <- s + xi
  }
  s
}
library(microbenchmark)
y <- $x
rbm <- microbenchmark(sum_r(y))
"""

RObject{VecSxp}
Unit: milliseconds
     expr      min       lq     mean median       uq      max neval
 sum_r(y) 17.46106 18.40698 19.24281 18.958 19.82638 26.13243   100


In [18]:
# store median runtime (in milliseconds)
@rget rbm # dataframe
benchmark_result["R loop"] = median(rbm[!, :time]) / 1e6

18.958004

### Python, builtin sum

Built in function `sum` in Python.

In [19]:
using PyCall
PyCall.pyversion

v"3.7.6"

In [20]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")
bm = @benchmark $pysum($x)

BenchmarkTools.Trial: 
  memory estimate:  352 bytes
  allocs estimate:  7
  --------------
  minimum time:     157.292 ms (0.00% GC)
  median time:      162.206 ms (0.00% GC)
  mean time:        162.168 ms (0.00% GC)
  maximum time:     173.395 ms (0.00% GC)
  --------------
  samples:          31
  evals/sample:     1

In [21]:
# store median runtime (in miliseconds)
benchmark_result["Python builtin"] = median(bm.times) / 1e6

162.205624

### Python, handwritten loop

In [22]:
using PyCall

py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

bm = @benchmark $sum_py($x)

BenchmarkTools.Trial: 
  memory estimate:  352 bytes
  allocs estimate:  7
  --------------
  minimum time:     212.511 ms (0.00% GC)
  median time:      221.725 ms (0.00% GC)
  mean time:        224.079 ms (0.00% GC)
  maximum time:     254.300 ms (0.00% GC)
  --------------
  samples:          23
  evals/sample:     1

In [23]:
# store median runtime (in miliseconds)
benchmark_result["Python loop"] = median(bm.times) / 1e6

221.724851

### Python, numpy

Numpy is the high-performance scientific computing library for Python.

In [24]:
# bring in sum function from Numpy 
numpy_sum = pyimport("numpy")."sum"

PyObject <function sum at 0x1515c0950>

In [25]:
bm = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 
  memory estimate:  352 bytes
  allocs estimate:  7
  --------------
  minimum time:     358.174 μs (0.00% GC)
  median time:      513.776 μs (0.00% GC)
  mean time:        559.687 μs (0.00% GC)
  maximum time:     1.719 ms (0.00% GC)
  --------------
  samples:          8814
  evals/sample:     1

In [26]:
# store median runtime (in miliseconds)
benchmark_result["Python numpy"] = median(bm.times) / 1e6

0.513776

Numpy performance is on a par with Julia build-in sum function. Both are about 3 times faster than C, possibly because of usage of [SIMD](https://en.wikipedia.org/wiki/SIMD).

### Summary

In [27]:
sort(collect(benchmark_result), by=x->x[2])

7-element Array{Pair{Any,Any},1}:
  "Julia builtin" => 0.444936
   "Python numpy" => 0.513776
      "R builtin" => 1.341087
              "C" => 1.500243
         "R loop" => 18.958004
 "Python builtin" => 162.205624
    "Python loop" => 221.724851

* `C` and `R builtin` are the baseline C performance (gold standard).

* `Python builtin` and `Python loop` are 80-100 fold slower than C because the loop is interpreted.

* `R loop` is about 30 folder slower than C and indicates the performance of bytecode generated by its compiler package (turned on by default since R v3.4.0 (Apr 2017)). 

* `Julia builtin` and `Python numpy` are 3-4 folder faster than C because of SIMD (???).